# Media

(Last updated: May 30, 2024)

Since around 2020, I have been keeping track of almost all media I consume - mostly books and games, but also concerts (but not CD's), movies, museums, and other forms of culture. Here, you can find a quick summary of what I've been up to

## Currently enjoying

In [26]:
import pandas as pd
df = pd.read_csv("media.csv")

verb_dict = {"Book":"read","Game":"played","Movie":"watched","Concert":"visited","Museum":"visited"}
print(f"Last {verb_dict.get(df['type'].iloc[-1], 'visited')}: {df['name'].iloc[-1]} by {df['creator'].iloc[-1]}")
score_dict = {10:'close to perfect!', 9:'fantastic!', 8:'great', 7:'good', 6:'okay', 5:'meh...', 4:'not great', 3:'bad', 2:'terrible', 1: 'abhorrent', 0:'the worst!'}
print(f"Which I thought was: {score_dict[round(df['score'].iloc[-1])]}")

Last played: Paper Mario: The Thousand-Year Door by Nintendo
Which I thought was: fantastic!


## All-time favorites

In [27]:
df3 = df.copy()
sample = df3[df3['score'] > 9.5].sample(5).reset_index()
print("Some works I can wholeheartedly recommend are:")
for i in range(len(sample)):
    print(f"* {sample['name'][i]} by {sample['creator'][i]}, which I rated at {sample['score'][i]}")

Some works I can wholeheartedly recommend are:
* The MANIAC by Benjamin Labatut, which I rated at 9.8
* Disco Elysium by ZA/UM, which I rated at 9.8
* Roshambo, of Waarom we Steen-Papier-Schaar Spelen by DraeCKI, which I rated at 9.9
* King Gizzard and the Wizard Lizard by King Gizzard and the Lizard Wizard, which I rated at 10.0
* Men at Arms by Terry Pratchett, which I rated at 9.8


## What?

In [28]:
import plotly.express as px
def rounder(score):
    score = float(round(score*2))/2
    #if score == 10:
    #    return "score≈" + str(9.9)
    #return "score≈" + str(score)
    return score

def counter(type, score):
    return len(df4[(df4['type'] == type) & (df4['score'] == score)])

df4 = df.copy()
#df4 = df4.groupby(['type', 'score']).size().rename_axis(['type', 'score']).reset_index(name='counts')
df4['counts'] = df4.apply(lambda x: counter(x.type, x.score), axis=1)
df4['scoreactual'] = df4['score'].apply(lambda x: rounder(x))
df4 = df4.sort_values(by=['type', 'score'])
fig = px.sunburst(df4, path=['type', 'scoreactual', 'name'], 
                  labels=[''], maxdepth=2,
                    hover_data={
                    })
fig.update_layout(margin={'t':5,'l':5,'b':5,'r':5}, showlegend=False)
fig.update_traces(sort=False, hovertemplate='<b>Amount: %{value} </b><br>%{label}')
fig.show()

## How much?

In [29]:
import plotly.express as px
import numpy as np

df2 = df.copy()
df2 = df2.groupby(['type', 'date']).size().rename_axis(['type', 'date']).reset_index(name='counts')
df2[['year','month']] = df2['date'].str.split('-',expand=True)
df2 = df2.drop(columns=['date'])
types = []; counts = []; years = []; months = []
for year in range(min(df2['year'].astype(int)), max(df2['year'].astype(int)) + 1):
    for month in range(1, 13):
        actualmonth = '0' * (2 - len(str(month))) + str(month)
        for type in df2['type'].unique():
            if not ((df2['year'] == str(year)) & (df2['month'] == actualmonth) & (df2['type'] == type)).any():
                types.append(type); counts.append(0); years.append(str(year)); months.append(actualmonth)
for year in range(min(df2['year'].astype(int)), max(df2['year'].astype(int)) + 1):
    for month in range(1, 13):
        actualmonth = '0' * (2 - len(str(month))) + str(month)
        types.append("All"); counts.append(df2[(df2['year'] == str(year)) & (df2['month'] == actualmonth)]['counts'].sum()); years.append(str(year)); months.append(actualmonth)
added_rows = pd.DataFrame({'type':types, 'counts':counts, 'year':years, 'month':months})
df2 = pd.concat([df2, added_rows], ignore_index=True)
types = []; counts = []; years = []; months = []
for month in range(1, 13):
    actualmonth = '0' * (2 - len(str(month))) + str(month)
    for type in df2['type'].unique():
        count = sum(df2[(df2['type'] == type) & (df2['month'] == actualmonth)]['counts']) / len(df2['year'].unique())
        count = count if not np.isnan(count) else 0
        types.append(type); counts.append(count); years.append("Mean"); months.append(actualmonth)
added_rows = pd.DataFrame({'type':types, 'counts':counts, 'year':years, 'month':months})
df2 = pd.concat([df2, added_rows], ignore_index=True)
df2 = df2.sort_values(by=['year', 'month'])
fig = px.line(df2, x='month', y='counts', color='year', animation_frame='type',
              color_discrete_map={"Mean": 'black'}, custom_data=['year'],
              labels={
                     "month": "Month",
                     "counts": "Amount",
                     "year": "Year"
                 },
                 hover_data={
                            'type':False,
                            'month':False,
                            'counts':False
                        })
fig.update_layout(margin={'t':10,'l':0,'b':0,'r':0}, yaxis_range=[0,15], xaxis_range=[0,11],
                  xaxis = dict(
                    tickmode = 'array',
                    tickvals = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
                    ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
                ), showlegend=False)
fig.update_traces(opacity=.4)
fig.update_traces(selector=dict(line_color="black"), opacity=1)
fig.update_traces(hovertemplate='%{customdata[0]}<extra></extra>')
fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 20, b=20)
fig["layout"].pop("updatemenus")
fig.show()


## Albums
One particular interest I have is collecting albums out of Rolling Stone's Top 500 Albums list. Below, you can see how many I already have (out of the 2020 list).

In [30]:
albums_df = pd.read_csv("albums.csv", sep=';')
albums_df['Rounded year'] = albums_df['Year'].apply(lambda x: 10 * round(int(x) / 10))
albums_df['Have'] = albums_df['Have'].apply(lambda x: 1 if x == "X" else x)
score = len(albums_df[(albums_df['Have'] == 1)]) / len(albums_df)
displays = int(score * 40)
print(f"Total:     [{"+" * displays}{" " * (40 - displays)}] {len(albums_df[(albums_df['Have'] == 1)])} out of {len(albums_df)}\n")
for year in sorted(albums_df['Rounded year'].unique()):
    score = len(albums_df[(albums_df['Rounded year'] == year) & (albums_df['Have'] == 1)]) / len(albums_df[albums_df['Rounded year'] == year])
    displays = int(score * 40)
    print(f"{year}-{year + 10}: [{"+" * displays}{" " * (40 - displays)}] {len(albums_df[(albums_df['Rounded year'] == year) & (albums_df['Have'] == 1)])} out of {len(albums_df[albums_df['Rounded year'] == year])}")

Total:     [++++++++                                ] 102 out of 500

1960-1970: [+                                       ] 1 out of 28
1970-1980: [+++++++++                               ] 33 out of 146
1980-1990: [++++++++                                ] 22 out of 106
1990-2000: [+++++++++                               ] 21 out of 88
2000-2010: [++++++++                                ] 17 out of 83
2010-2020: [+++++++++                               ] 7 out of 31
2020-2030: [++                                      ] 1 out of 18


In [31]:
albums_df['Have'] = albums_df['Have'].cumsum()
albums_df = albums_df[albums_df['Have'].notnull()]
fig = px.line(albums_df, x='#', y='Have',
              labels={
                     "#": "Number of albums in top 500",
                     "Have": "Amount in collection",
                 },)
fig.update_layout(margin={'t':10,'l':0,'b':0,'r':10}, showlegend=False)
fig.show()